In [21]:
import pytesseract
from pytesseract import image_to_string
from PIL import Image
from selenium import webdriver
import urllib
import time
from bs4 import BeautifulSoup
import requests
import urllib.request
import pandas as pd
from selenium.webdriver.common.keys import Keys

In [15]:
! pip install --user pytesseract

In [17]:
! pip install --user xlrd

In [4]:
! pip install --user selenium

In [6]:
def get_captcha_text(location, size):
    driver.get('http://www.mca.gov.in/mcafoportal/showIndexOfCharges.do')
    try:
        driver.find_element_by_xpath('/html/body/div[1]/div[6]/div[1]/section/div[2]/a').click()
        time.sleep(1)
        driver.save_screenshot('screenshot.png')
        pytesseract.pytesseract.tesseract_cmd = r"C:\Users\Administrator\Desktop\Tesseract-OCR\tesseract" # specify the path of tesseract.exe
        im = Image.open('screenshot.png')  # uses PIL library to open image in memory

        left = location['x']
        top = location['y']
        right = location['x'] + size['width']
        bottom = location['y'] + size['height']

        im = im.crop((left, top, right, bottom))  # defines crop points
        im.save('screenshot1.png')
        captcha_text = image_to_string(Image.open('screenshot1.png'), config="-c tessedit_char_whitelist=abcdefghijklmnopqrstuvwxyz")
        captcha_text= captcha_text.lower()
        return captcha_text
    except:
        driver.save_screenshot('screenshot.png')
        pytesseract.pytesseract.tesseract_cmd = r"C:\Users\Administrator\Desktop\Tesseract-OCR\tesseract" #specify the path of tesseract.exe

        im = Image.open('screenshot.png')  # uses PIL library to open image in memory

        left = location['x']
        top = location['y']
        right = location['x'] + size['width']
        bottom = location['y'] + size['height']

        im = im.crop((left, top, right, bottom))  # defines crop points
        im.save('screenshot1.png')
        captcha_text = image_to_string(Image.open('screenshot1.png'), config="-c tessedit_char_whitelist=abcdefghijklmnopqrstuvwxyz")
        captcha_text = captcha_text.lower()
        return captcha_text


def something(cin):
    recap = get_captcha_text(location, size)
    driver.find_element_by_xpath(
        '/html/body/div[1]/div[6]/div[1]/section/form/table/tbody/tr[2]/td[2]/input').send_keys(cin)
    driver.find_element_by_xpath(
        '/html/body/div[1]/div[6]/div[1]/section/form/table/tbody/tr[4]/td[2]/input').send_keys(recap)
    time.sleep(1)
    driver.find_element_by_xpath(
        '/html/body/div[1]/div[6]/div[1]/section/form/table/tbody/tr[5]/td[2]/input[1]').click()
    time.sleep(2)


def submit(cin):
    try:
        something(cin)
        while driver.find_element_by_xpath('//*[@id="overlayCnt"]/ul/li').text == 'Enter valid Letters shown.':
            something(cin)
    except:
        pass

In [58]:
driver = webdriver.Chrome(r"C:\Users\Administrator\Desktop\chromedriver_win32\chromedriver.exe") # specify the path of chrome driver.
driver.get('http://www.mca.gov.in/mcafoportal/showIndexOfCharges.do')


In [59]:
element = driver.find_element_by_xpath('/html/body/div[1]/div[6]/div[1]/section/form/table/tbody/tr[3]/td[2]/img')
location = element.location
size = element.size
data = pd.read_csv('listed_companies.csv') # the list of CINs and to search. specify the path if ithe code is running in a different directory
cins = data['mca_cin'].to_list()
company_name = data['mca_name'].to_list()
df_columns = ['SNo', 'SRN', 'Charge_Id', 'Charge_Holder_Name', 'Date_of_Creation', 'Date_of_Modification', 'Date_of_Satisfaction', 'Amount', 'Address']


In [61]:
A_data =  pd.DataFrame()
idx_noChrg = []
for i in range(len(cins)):
    try:
        print(company_name[i])
        submit(cins[i])
        driver.find_element_by_xpath('/html/body/div[1]/div[6]/div[1]/section/form/table/tbody/tr[3]/td[2]/a/img').click()
        time.sleep(2)
        page = driver.find_element_by_xpath('//*[@id="charges_info"]')
        num = int(page.text.strip(' entries')[-2:])
        q,r = divmod(num ,10)
        
        for k in range(q+1):
            html = driver.page_source
            soup = BeautifulSoup(html)

            table = soup.findAll('tr', {'class': ['odd','even','table-header']})
            df = pd.DataFrame()
            for j in range(len(table)-2):
                row = table[j+2].text.strip('\n').split('\n')
                df[j] = row

            df = df.transpose()
            df.columns = df_columns
            df['cin'] = cins[i]
            df['company_name'] = company_name[i]

            A_data = A_data.append(df)
            A_data.to_csv("mca_data.csv")
            driver.find_element_by_xpath('//*[@id="charges_next"]').click()

    except:
        idx_noChrg.append(company_name[i])
        pass

'K' STEAMSHIP AGENCIES PVT. LTD.
20 MICRONS LTD.
20TH CENTURY ENGG. LTD.
GREENSTONE INVESTMENTS PVT. LTD.
20TH CENTURY FINANCE CORPN. LTD. [MERGED]


In [ ]:
DF = pd.DataFrame(idx_noChrg)
DF.to_csv("not_available.csv")